In [2]:
# Dependencies
import sys
import time
import json
import requests
import importlib
import numpy as np
import pandas as pd
from pprint import pprint
import matplotlib.pyplot as plt
from config import gkey, zkey, zws_id, akey

In [3]:
# get list of zipcodes
%run ./write_zips.ipynb
import zipcodes
importlib.reload(zipcodes)
zip_codes = zipcodes.ziplist

Target city: atlanta
Search radius: 1


In [4]:
# get http request headers
def getHeaders():
    headers = {
        "apikey":akey,
        "accept":"application/json"
    }
    return headers


# GET /property/address via zip code
def fetch(url, params):
    headers = getHeaders()
    res = requests.get(url, headers=headers, params=params).json()
    return res


# show load bar and wait for next request
def wait(t):    
    sys.stdout.write('.')
    sys.stdout.flush()
    time.sleep(t)

In [63]:
url = "https://search.onboard-apis.com/propertyapi/v1.0.0/property/address?"
property_results = []
prop_types = ['RESIDENTIAL (NEC)', 'CONDOMINIUM', 
              'APARTMENT', 'CABIN', 'DUPLEX', 'MOBILE HOME',
              'TRIPLEX', 'TOWNHOUSE/ROWHOUSE', 'MULTI FAMILY DWELLING']

sys.stdout.write('working..')

for z in zip_codes:
    percent_complete = f"{round((len(property_results)/len(prop_types)*100),2)}%"  # abstract to function
#     getLoadComplete(propert_results, prop_types)
    for pt in prop_types:
        params = {
            "postalcode": z,
            "propertysubtype": 'RESIDENTIAL',
            "pagesize": "100"
        }

        res = fetch(url, params)

        property_results.append(res)
        wait(8)
    
    
property_results

{'status': {'version': '1.0.0',
  'code': -5,
  'msg': 'Invalid Parameter in Request',
  'total': 0,
  'page': 0,
  'pagesize': 0}}

In [6]:
# get properties for each zip code (10 queries/min)
url = "https://search.onboard-apis.com/propertyapi/v1.0.0/property/address?"
property_results = []
prop_types = ['RESIDENTIAL (NEC)', 'CONDOMINIUM', 
              'APARTMENT', 'CABIN', 'DUPLEX', 'MOBILE HOME',
              'TRIPLEX', 'TOWNHOUSE/ROWHOUSE', 'MULTI FAMILY DWELLING']

sys.stdout.write('working..')

for z in zip_codes:
    percent_complete = f"{round((len(property_results)/len(prop_types)*100),2)}%"  # abstract to function
#     getLoadComplete(propert_results, prop_types)
    for pt in prop_types:
        params = {
            "postalcode": z,
            "propertytype": pt,
            "pagesize": "100"
        }

        res = fetch(url, params)

        property_results.append(res)
        wait(8)

working.............................................................................................................................................................................

In [7]:
# get attomIds from property results
properties = []
attomIds = []

for p in property_results:
    for q in p['property']:
        properties.append(q)

props_df = pd.DataFrame(properties)
attomId_series = props_df['identifier']

for a in attomId_series:
    attomIds.append(a['attomId'])

In [8]:
# get home values for each property (10 queries/min)
url = "https://search.onboard-apis.com/propertyapi/v1.0.0/property/basicprofile?"
hv_results = []

sys.stdout.write("working..")

for a in attomIds:
    params = {
        "attomId": a,
        "page":"1",
        "pagesize":"100"
    }

    idres = fetch(url, params)
    hv_results.append(idres)    
    wait(8)

working...................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [1]:
# for r in hv_results:
#     pprint(r['property'][0])

# pd.read_json('results.json')

In [69]:
propslist = []

for r in hv_results:
    address = r['property'][0]['address']['oneLine']
    mkt_val = (r['property'][0]['assessment']['market']['mktTtlValue'])
    prop_type = r['property'][0]['summary']['propType']
    prop_sub_type = r['property'][0]['summary']['propSubType']
    propslist.append({"address":address,"prop_type":prop_type,"prop_sub_type":prop_sub_type,"price":mkt_val})
    
properties_df = pd.DataFrame(propslist)
properties_df['prop_sub_type'].value_counts()
properties_df['prop_type'].value_counts()

# properties_df = 
# properties_df.groupby('prop_type').sum()/len(propslist)
# properties_df.style.format({'price':'${:,.2f}'})

CONDOMINIUM          200
APARTMENT            115
DUPLEX               101
TRIPLEX               28
RESIDENTIAL (NEC)     21
Name: prop_type, dtype: int64

In [27]:
prop_res = pd.DataFrame(hv_results)
properties = hv_results['property']
zcmkv = []

for i in range(len(props)):
    zcmkv.append({"zipcode":props[i][0]['address']['postal1'], "avg_mkt_val":int(props[i][0]['assessment']['market']['mktTtlValue'])})

zips_mktvals_df = pd.DataFrame(zcmkv).set_index('zipcode')
mkt_avg = zips_mktvals_df.groupby('zipcode').sum()/len(props)

mkt_avg_df = pd.DataFrame(mkt_avg)
mkt_avg_df = mkt_avg_df.reset_index()
mkt_avg_df.style.format({'avg_mkt_val':'${:,.2f}'})

,zipcode,avg_mkt_val
0,30303,"$520,996.80"
1,30312,"$876,251.55"


In [87]:
pd.DataFrame(hv_results[0]['property'][0])

,identifier,lot,area,address,location,summary,utilities,sale,building,assessment,vintage
absenteeInd,NaN,NaN,NaN,NaN,NaN,OWNER OCCUPIED,NaN,NaN,NaN,NaN,NaN
accuracy,NaN,NaN,NaN,NaN,Street,NaN,NaN,NaN,NaN,NaN,NaN
apn,14 002300120414,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
appraised,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'apprImprValue': 0.0, 'apprLandValue': 0.0, '...",NaN
assessed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'assdImprValue': 80240.0, 'assdLandValue': 19...",NaN
attomId,24949968,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
censusBlockGroup,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
censusTractIdent,NaN,NaN,6900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
construction,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'frameType': 'TYPE UNKNOWN'},NaN,NaN
coolingType,NaN,NaN,NaN,NaN,NaN,NaN,AC.CENTRAL,NaN,NaN,NaN,NaN
